In [9]:
#加载飞桨和相关类库
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import os
import numpy as np
import matplotlib.pyplot as plt
from paddle.vision.transforms import Normalize


In [10]:
# 设置数据读取器，API自动读取MNIST数据训练集
train_dataset = paddle.vision.datasets.MNIST(mode='train')

In [3]:
# 定义mnist数据识别网络结构，同房价预测网络
class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__()
        
        # 定义一层全连接层，输出维度是1
        self.fc = paddle.nn.Linear(in_features=784, out_features=1)
        
    # 定义网络结构的前向计算过程
    def forward(self, inputs):
        outputs = self.fc(inputs)
        return outputs

In [4]:
# 图像归一化函数，将数据范围为[0, 255]的图像归一化到[0, 1]
def norm_img(img):
    # 验证传入数据格式是否正确，img的shape为[batch_size, 28, 28]
    assert len(img.shape) == 3
    batch_size, img_h, img_w = img.shape[0], img.shape[1], img.shape[2]
    # 归一化图像数据
    img = img / 255
    # 将图像形式reshape为[batch_size, 784]
    img = paddle.reshape(img, [batch_size, img_h*img_w])
    
    return img

In [5]:
import paddle
# 确保从paddle.vision.datasets.MNIST中加载的图像数据是np.ndarray类型
paddle.vision.set_image_backend('cv2')

# 声明网络结构
model = MNIST()

def train(model):
    # 启动训练模式
    model.train()
    # 加载训练集 batch_size 设为 16
    train_loader = paddle.io.DataLoader(paddle.vision.datasets.MNIST(mode='train'), 
                                        batch_size=16, 
                                        shuffle=True)
    # 定义优化器，使用随机梯度下降SGD优化器，学习率设置为0.001
    opt = paddle.optimizer.SGD(learning_rate=0.001, parameters=model.parameters())
    EPOCH_NUM = 10
    for epoch in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            images = norm_img(data[0]).astype('float32')
            labels = data[1].astype('float32')
            
            #前向计算的过程
            predicts = model(images)
            
            # 计算损失
            loss = F.square_error_cost(predicts, labels)
            avg_loss = paddle.mean(loss)
            
            #每训练了1000批次的数据，打印下当前Loss的情况
            if batch_id % 1000 == 0:
                print("epoch_id: {}, batch_id: {}, loss is: {}".format(epoch, batch_id, avg_loss.numpy()))
            
            #后向传播，更新参数的过程
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
            
train(model)
paddle.save(model.state_dict(), './mnist.pdparams')

epoch_id: 0, batch_id: 0, loss is: 20.11361312866211
epoch_id: 0, batch_id: 1000, loss is: 3.081111431121826
epoch_id: 0, batch_id: 2000, loss is: 3.2629332542419434
epoch_id: 0, batch_id: 3000, loss is: 4.0055012702941895
epoch_id: 1, batch_id: 0, loss is: 3.611865997314453
epoch_id: 1, batch_id: 1000, loss is: 2.583967447280884
epoch_id: 1, batch_id: 2000, loss is: 2.9955530166625977
epoch_id: 1, batch_id: 3000, loss is: 4.041898727416992
epoch_id: 2, batch_id: 0, loss is: 2.287470817565918
epoch_id: 2, batch_id: 1000, loss is: 3.9004974365234375
epoch_id: 2, batch_id: 2000, loss is: 2.0123229026794434
epoch_id: 2, batch_id: 3000, loss is: 2.329176425933838
epoch_id: 3, batch_id: 0, loss is: 1.519679069519043
epoch_id: 3, batch_id: 1000, loss is: 4.726850986480713
epoch_id: 3, batch_id: 2000, loss is: 2.2296509742736816
epoch_id: 3, batch_id: 3000, loss is: 3.574422836303711
epoch_id: 4, batch_id: 0, loss is: 4.445947170257568
epoch_id: 4, batch_id: 1000, loss is: 2.2102999687194824


In [24]:
# 加载模型
layer = paddle.nn.Conv2D(1, 10, 3)
layer.set_state_dict(paddle.load('./mnist.pdparams'))
transform = Normalize(mean=[127.5], std=[127.5], data_format="CHW")
test_dataset = paddle.vision.datasets.MNIST(mode="test",transform=transform)
img, label = test_dataset[0]
# 将图片shape从1*28*28变为1*1*28*28，增加一个batch维度，以匹配模型输入格式要求
img_batch = np.expand_dims(img.astype("float32"), axis=0)
# 执行推理并打印结果，此处predict_batch返回的是一个list，取出其中数据获得预测结果
out = layer.forward(paddle.to_tensor(img_batch))
pred_label = out.argmax()
print("true label: {}, pred label: {}".format(label[0], pred_label))
# 可视化图片
from matplotlib import pyplot as plt

plt.imshow(img[0])

d:\anaconda3\envs\paddle_24\lib\site-packages\paddle\nn\layer\layers.py:2084: UserWarning: Skip loading for weight. weight is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
d:\anaconda3\envs\paddle_24\lib\site-packages\paddle\nn\layer\layers.py:2084: UserWarning: Skip loading for bias. bias is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))


true label: 7, pred label: Tensor(shape=[], dtype=int64, place=Place(cpu), stop_gradient=True,
       6296)


: 